Sto considerando di usare i seguenti modelli di ML:
1. ResNet.
2. GoogLeNet.
3. AlexNet.

In [1]:
import pathlib

data_dir = pathlib.Path('data/chest_xray/train')
total = len(list(data_dir.glob('**/*.jpeg')))
bacteria = len(list(data_dir.glob('**/BACTERIA-*.jpeg')))
virus = len(list(data_dir.glob('**/VIRUS-*.jpeg')))
P = len(list(data_dir.glob('**/NORMAL-*.jpeg')))
N = total - P
print(f'#training set = {total}')
print(f'#positives = {P}')
print(f'#negatives = {N} (bacteria = {bacteria}, virus = {virus})')

#training set = 5232
#positives = 1349
#negatives = 3883 (bacteria = 2538, virus = 1345)


Le immagini nel training set sono 5232 di cui: 1349 appartenenti alla classe NORMAL mentre 3883 alla classe PNEUMONIA.
Sto considerando diverse strategie per aumentare la dimensione del dataset:
1. Trasformazioni geometriche: specchiare, ruotare, tagliare, allungare.
   a. Specchiare e ruotare non mi sembrano che possano essere molto utili per questa tipologia di problema. Similmente anche
      il ritagliare l'immagine potrebbe rimuovere porzioni di immagini che potrebbero rivelarsi fondamentali per individuare o meno
      la polmonite.
   b. Tag
2. Modificare la luminosità.

In [2]:
from PIL import Image
images = list(map(lambda i: Image.open(str(i)), data_dir.glob('**/*.jpeg')))

In [3]:
import matplotlib.pyplot as plt
import numpy as np

def visualize(original, augmented):
  original = 255 - np.array(original)
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original, cmap='gray')
  
  augmented = 255 - np.array(augmented)
  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented, cmap='gray')

2025-01-21 12:53:13.431591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-21 12:53:13.468742: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-21 12:53:13.478382: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 12:53:13.570328: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


È necessario ridimensionare le immagini per:
- Utilizzare meno risorse computazionali.
- Velocizzare sia addestramento che inferenza.
Al momento sto considerando le diverse strategie per il ridimensionamento delle immagini:
1. Ridimensionare tutte le immagini a 133x133 dove 133 è l'altezza minima tra tutte le immagini.
2. Ridimensionare tutte le immagini a 290x290 dove 290 è l'altezza media delle immagini